In [ ]:
#Temporary example

import plotly.express as px

# simple example data
data = {
    "category": ["A", "B", "C", "D"],
    "value": [10, 25, 15, 30],
}

# create bar chart
fig = px.bar(
    data,
    x="category",
    y="value",
    title="Ein einfaches Plotly-Beispiel",
    labels={
        "category": "Kategorie",
        "value": "Wert"
    }
)

# show figure
fig.show()


# 🚀 Sampling und Filterung des Korpus

Für die weitere Analyse konzentrieren wir uns auf den Zeitraum **1800–1900**.
Zudem möchten wir sicherstellen, dass jedes Jahrzehnt annähernd gleich stark vertreten ist.|

In [ ]:
import pandas as pd
import plotly.graph_objects as go

In [ ]:
meta = pd.read_csv('../metadata/metadata_corpus-german_language_fiction.csv')
meta.head()

In [ ]:
meta['decade'] = (meta['year']//10*10).astype(str)
decade_counts = meta['decade'].value_counts().sort_index()

Filterung

In [ ]:
subset = meta.query("1820 <= year < 1900")
subset.shape

In [ ]:
# pro Jahrzehnt 50 Texte zufällig ziehen -- no 1 (random state = 42)
subset_decades = (
    subset.groupby(subset['year']//10*10)
    .apply(lambda x: x.sample(n=min(50, len(x)), random_state=42))
    .reset_index(drop=True)
)

In [ ]:
subset_decades

In [ ]:
# Anzahl der Texte pro Jahrzehnt
def summarize_texts_per_decade(df, decade_column):
    """Return per-yearcounts and summary stats for the given year column."""
    bins = df[decade_column].dropna()
    if bins.empty:
        raise ValueError(f"No year values found in column '{decade_column}'.")
    bins = bins.astype(int)
    counts = bins.value_counts().sort_index()
    stats = counts.agg(['mean', 'max', 'min']).rename({'mean': 'avg_per_year'})
    return counts, stats

subset_decades['decade'] = (subset_decades['year'] // 10) * 10
decade_counts, decade_stats = summarize_texts_per_decade(subset_decades, 'decade')

print("Textanzahl der Texte im Subset pro Jahrzehnt:")
print(decade_stats)

In [ ]:
# Visualisierung der Dekadenverteilung (Textanzahl pro Dekade)
fig_decade = px.bar(
    x=decade_counts.index.astype(str),
    y=decade_counts.values,
    labels={
        "x": "Jahrzehnt",
        "y": "Anzahl Texte"
    },
    title="Zeitliche Verteilung der Texte im 'Corpus of German-Language Fiction' pro Jahrzehnt"
)

fig_decade.update_layout(
    height=350,
    margin=dict(l=40, r=40, t=60, b=40),
)

fig_decade.show()

In [ ]:
subset_decades.to_csv('../metadata/metadata_corpus-german_language_fiction_1820-1900_50-per-decade.csv', index=False)

### Create the alternative sample with different random state

In [ ]:
# pro Jahrzehnt 50 Texte zufällig ziehen -- no 2 (random state = 31415)
subset_decades_alt = (
    subset.groupby(subset['year']//10*10)
    .apply(lambda x: x.sample(n=min(50, len(x)), random_state=31415))
    .reset_index(drop=True)
)

subset_decades_alt['decade'] = (subset_decades_alt['year'] // 10) * 10

In [ ]:
subset_decades_alt.to_csv('../metadata/metadata_corpus-german_language_fiction_1820-1900_50-per-decade_ALT.csv', index=False)

### Compare the overlap between the samples

In [ ]:
# convert ID columns to sets
ids = set(subset_decades["ID"])
ids_alt = set(subset_decades_alt["ID"])

only_main = len(ids - ids_alt)
only_alt = len(ids_alt - ids)
in_both = len(ids & ids_alt)

print(f"Subset overlap: {in_both}")
print(f"Only subset_decades: {only_main}")
print(f"Only subset_decades_alt: {only_alt}")

# --- Create Plotly figure ---
fig = go.Figure()

# Left circle (subset_decades)
fig.add_shape(
    type="circle",
    xref="x",
    yref="y",
    x0=0,
    y0=0,
    x1=2,
    y1=2,
    fillcolor="rgba(31, 119, 180, 0.4)",
    line_color="rgba(31, 119, 180, 1)",
)

# Right circle (subset_decades_alt)
fig.add_shape(
    type="circle",
    xref="x",
    yref="y",
    x0=1,
    y0=0,
    x1=3,
    y1=2,
    fillcolor="rgba(255, 127, 14, 0.4)",
    line_color="rgba(255, 127, 14, 1)",
)

# --- Annotations for counts ---
fig.add_annotation(
    x=0.7, y=1,
    text=str(only_main),
    showarrow=False,
    font=dict(size=18)
)

fig.add_annotation(
    x=2.3, y=1,
    text=str(only_alt),
    showarrow=False,
    font=dict(size=18)
)

fig.add_annotation(
    x=1.5, y=1,
    text=str(in_both),
    showarrow=False,
    font=dict(size=18, color="black")
)

# --- Set labels ---
fig.add_annotation(
    x=0.5, y=2.15,
    text="subset_decades",
    showarrow=False,
    font=dict(size=14)
)

fig.add_annotation(
    x=2.5, y=2.15,
    text="subset_decades_alt",
    showarrow=False,
    font=dict(size=14)
)

# --- Layout ---
fig.update_layout(
    title="Overlap of Corpus Samples (by ID)",
    xaxis=dict(visible=False, range=[-0.2, 3.2]),
    yaxis=dict(visible=False, range=[-0.2, 2.6], scaleanchor="x"),
    height=500,
    width=500,
    margin=dict(l=20, r=20, t=60, b=20),
)

fig.show()


## Reflexion: Auswahl als epistemische Entscheidung

Die Entscheidung für ein bestimmtes Korpus ist nie neutral.
Sie bestimmt, **welche literarischen Stimmen, Räume und historischen Kontexte** in der Analyse sichtbar werden.
Die bewusste Reflexion über diese Auswahl ist ein zentraler Bestandteil datenbasierter geisteswissenschaftlicher Forschung.

```{admonition} Merksatz
:class: keypoint
Mit der Auswahl eines Korpus konstruieren wir das epistemische Objekt unserer Forschung – und damit die Grenzen dessen, was wir erkennen können.
```